In [1]:
%load_ext autoreload

In [2]:
%autoreload
from pipeline import DataPipeline, BuilingIdsEnum
pipe = DataPipeline()

from ipywidgets import interact, IntSlider
from matplotlib.widgets import RangeSlider
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)

In [3]:
main_building = pipe.get_import_data_for_building(BuilingIdsEnum.MAIN.value)
my_path = Path().resolve().parent / 'data' / 'prediciton1.csv'
# open the csv and read it to a dataframe
prediciton1_df = pd.read_csv(my_path)

# merge the prediciotn data with the main building data they are indexed correctly.
main_building = main_building.merge(prediciton1_df, left_index=True, right_index=True)
main_building.rename(columns={'value_y': 'prediction', 'value_x': 'actual'}, inplace=True)

# 	actucal	prediction
df_display = pd.DataFrame(main_building[['actual', 'prediction', 'timestamp']])
df_display.set_index('timestamp', inplace=True)
df_display

,actual,prediction
timestamp,,
2023-07-01 00:00:00,27.5,19.054262
2023-07-01 01:00:00,27.1,46.911950
2023-07-01 02:00:00,26.8,57.422485
2023-07-01 03:00:00,25.9,48.097515
2023-07-01 04:00:00,26.4,27.604540
...,...,...
2023-09-10 15:00:00,0.4,47.724100
2023-09-10 16:00:00,5.1,35.531840
2023-09-10 17:00:00,14.1,41.579346


In [4]:
import plotly.graph_objects as go
import pandas as pd

# Assuming df_display is your DataFrame and already has 'timestamp' as its index
# If not, set it: 


# Create the figure
fig = go.Figure()

# Add trace for actual energy usage
fig.add_trace(go.Scatter(x=df_display.index, y=df_display['actual'], 
                         mode='lines', name='Actual Energy (kWh)', 
                          line=dict(color='rgba(255, 99, 71, 0.9)')))
                         

# Add trace for predicted energy usage
fig.add_trace(go.Scatter(x=df_display.index, y=df_display['prediction'], 
                         mode='lines', name='Forecasted Energy (kWh)', 
                          line=dict(color='rgba(30, 144, 255, 0.9)'))) 
                        #  line=dict(color='blue')))

# Add fill between actual energy usage and baseline (y=0)
fig.add_trace(go.Scatter(
    x=df_display.index.tolist(),
    y=df_display['actual'].tolist(),
    fill='tozeroy',  # Fill to the x-axis (y=0)
    fillcolor='rgba(255, 160, 122, 0.3)',  # Lighter pastel red with opacity 0.3
    opacity=0.3,
    line=dict(color='rgba(255, 99, 71, 0)'),  # Hide the fill boundary
    showlegend=False
))

# Add fill between forecasted energy usage and baseline (y=0)
fig.add_trace(go.Scatter(
    x=df_display.index.tolist(),
    y=df_display['prediction'].tolist(),
    fill='tozeroy',  # Fill to the x-axis (y=0)
    fillcolor='rgba(135, 206, 250, 0.3)',  # Lighter pastel blue with opacity 0.3
    opacity=0.3,
    line=dict(color='rgba(30, 144, 255, 0)'),  # Hide the fill boundary
    showlegend=False
))

# Add layout details
fig.update_layout(
    title='Actual vs Forecasted Energy Usage',
    xaxis_title='Date',
    yaxis_title='Energy Usage (kWh)',
    hovermode='x unified',
    showlegend=True,
    height=600
)

# Add range slider and range selector for zoom functionality
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1d", step="day", stepmode="backward"),
                dict(count=7, label="1w", step="day", stepmode="backward"),
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(visible=True),
        type="date"
    )
)

# Show the plot
fig.show()